In [ ]:
import pandas as pd
import numpy as np

from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms

from statsmodels.tsa.ar_model import AutoReg, ar_select_order

from arch import arch_model

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import math

import quandl
quandl.ApiConfig.api_key = "q2DtgvtfT_ZV2DbsFyCF"

In [ ]:
## La siguiente función puede utilizarse para analizar tres gráficos: la serie, la función 
## autocorrelación y la función de autocorrelación parcial

def tsplot(y, lags=None, figsize=(15, 10), style='bmh', titulo = 'Time Series Analysis Plots'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        #qq_ax = plt.subplot2grid(layout, (2, 0))
        #pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(titulo)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.01)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.01)
        #sm.qqplot(y, line='s', ax=qq_ax)
        #qq_ax.set_title('QQ Plot')        
        #scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
rates_1yr = pd.read_csv("Data/w-gs1yr.csv")
rates_3yr = pd.read_csv("Data/w-gs3yr.csv")

IBM_VW_EW_SyP_index = pd.read_csv("Data/m-ibm3dx2608.csv")
IBM_VW_EW_SyP_index['date'] = pd.to_datetime(IBM_VW_EW_SyP_index['date'].astype(str), format ='%Y%m%d' )

GNP = pd.read_csv("Data/dgnp82.csv")
S_P_index_d = pd.read_csv("Data/d-sp55008.csv")
S_P500_index = pd.read_csv("Data/S_P500_index.csv")

VW_Index_rets = pd.read_csv("Data/VW_Index_rets.csv")

FX_logret_mark_dollar = pd.read_csv("Data/FX_logret_Mark_Dollar.csv")

Intel_returns = pd.read_csv("Data/Intel_returns.csv")
Intel_returns['date'] = pd.to_datetime(Intel_returns['date'].astype(str), format='%Y%m%d')

## Heteroscedasticidad Condicional

In [ ]:
# Simulación de un ARCH(1)
# Var(yt) = a_0 + a_1*y{t-1}**2
# si a_1 está entre 0 y 1 entonces es un ruido blanco.

np.random.seed(13)

a0 = 2
a1 = .5

y = w = np.random.normal(size=1000)
#Y = np.empty_like(y)

for t in range(len(y)):
    y[t] = w[t] * np.sqrt((a0 + a1*y[t-1]**2))

# un ARCH(1) se asemeja a un ruido blanco
tsplot(y, lags=30, titulo = "ARCH(1)")

In [ ]:
tsplot(y**2, lags=30, titulo = 'Valores al cuadrado')

Notemos que la serie temporal parece ruido blanco, pero para los cuadrados se puede observar una autocorrelación significativa en el lag 1.
Así como podemos modelarlo como un AR(p), puede utilizarse un modelo MA(q) o ARMA(p,q), lo cual conduce a los modelos ARCH generalizados, conocidos como GARCH

##  Generalized Autoregressive Conditionally Heteroskedastic Models - GARCH(p,q)
 
 (Modelos Autorregresivos generalizado condicional heteroscedástico)

$$\sigma_t^2=\alpha_0+\sum_{i=1}^{p}\alpha_i\epsilon_{t-i}^2+\sum_{j=1}^{q}\beta_j\sigma_{t-j}^2$$


In [ ]:
# Simulación de un proceso  GARCH(1, 1)

np.random.seed(2)

a0 = 0.2
a1 = 0.5
b1 = 0.3

n = 10000
w = np.random.normal(size=n)
eps = np.zeros_like(w)
sigsq = np.zeros_like(w)

for i in range(1, n):
    sigsq[i] = a0 + a1*(eps[i-1]**2) + b1*sigsq[i-1]
    eps[i] = w[i] * np.sqrt(sigsq[i])

_ = tsplot(eps, lags=30, titulo = "GARCH(1,1)")
#_ = tsplot(eps ** 2, lags=30, titulo = "GARCH(1,1) al cuadrado")

In [ ]:
am = arch_model(eps)
res = am.fit(update_freq=5)
print(res.summary())

Retomamos de la Notebook Nb 2 el caso de la serie S_p500_index. 


In [ ]:
data = S_P500_index["Monthly excess returns"]
mdl = sm.tsa.arima.ARIMA(data, order = (3,0,0))
res = mdl.fit()
print(res.summary())


In [ ]:
sel = ar_select_order(data, 8, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:
tsplot(res.resid ** 2, lags=30, titulo = "residuos")

In [ ]:
am = arch_model(res.resid, rescale = False)
res = am.fit(update_freq=10)
print(res.summary())

In [ ]:
### Posible modelo AR(3) + GARCH(1,1)

np.random.seed(2)

a0 = 0.2
a1 = 0.1698
b1 = 0.8132

n = len(data)


w = np.random.normal(size=n, loc = 0.0, scale = 1.0/10.)
eps = np.zeros_like(w)
sigsq = np.zeros_like(w)
#eps = np.random.normal(size=n)
#sigsq = np.random.normal(size=n)

for i in range(1, n):
    sigsq[i] = a0 + a1*(eps[i-1]**2) + b1*sigsq[i-1]
    eps[i] = w[i] * math.sqrt(sigsq[i])

x = np.zeros(len(data))
x[0] = data[0]
x[1] = data[1]
x[2] = data[2]
for i in range(2, n):
    x[i] = 0.0062 + 0.0891 * x[i-1] - 0.0237*x[i-2] - 0.1228 * x[i-3]  + eps[i]
